<a href="https://colab.research.google.com/github/Amt15/Pyspark/blob/main/how_to_handle_multi_delimiter_files_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark -q
!pip install findspark -q

     |████████████████████████████████| 281.3 MB 37 kB/s 
     |████████████████████████████████| 199 kB 50.0 MB/s 


In [2]:
import findspark
findspark.init()
findspark.find()

'/usr/local/lib/python3.7/dist-packages/pyspark'

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .appName("how to handle double delimiter multi delimiters files data")\
        .master("local[*]")\
        .enableHiveSupport()\
        .getOrCreate()

In [5]:
df = spark.read.csv("/content/double_pipe.csv",header=True,sep="|")
df.show()

+---+----+------+----+---------+
| id| _c1|  name| _c3|      loc|
+---+----+------+----+---------+
|  1|null|  ravi|null|    patna|
|  2|null|   ram|null|bangalore|
|  3|null|prasad|null|  chennai|
|  4|null|   sam|null|     pune|
|  5|null| shyam|null|    delhi|
+---+----+------+----+---------+



In [6]:
df = spark.read.csv("/content/double_pipe.csv",header=True,sep="||")
df.show()

+---+------+---------+
| id|  name|      loc|
+---+------+---------+
|  1|  ravi|    patna|
|  2|   ram|bangalore|
|  3|prasad|  chennai|
|  4|   sam|     pune|
|  5| shyam|    delhi|
+---+------+---------+



In [23]:
df1 = spark.read.csv("/content/multi_sep.csv",header=True)
df1.show()

+---+------+---------+-----------+
| id|  name|      loc|      marks|
+---+------+---------+-----------+
|  1|  ravi|    patna|34|45|55|65|
|  2|   ram|bangalore|38|40|59|70|
|  3|prasad|  chennai|37|42|54|77|
|  4|   sam|     pune|36|46|52|76|
|  5| shyam|    delhi|35|47|50|80|
+---+------+---------+-----------+



In [24]:
from pyspark.sql.functions import split,col

df1.withColumn("marks_split",split(col("marks"),"|")).show(truncate=False)

+---+------+---------+-----------+-----------------------------------+
|id |name  |loc      |marks      |marks_split                        |
+---+------+---------+-----------+-----------------------------------+
|1  |ravi  |patna    |34|45|55|65|[3, 4, |, 4, 5, |, 5, 5, |, 6, 5, ]|
|2  |ram   |bangalore|38|40|59|70|[3, 8, |, 4, 0, |, 5, 9, |, 7, 0, ]|
|3  |prasad|chennai  |37|42|54|77|[3, 7, |, 4, 2, |, 5, 4, |, 7, 7, ]|
|4  |sam   |pune     |36|46|52|76|[3, 6, |, 4, 6, |, 5, 2, |, 7, 6, ]|
|5  |shyam |delhi    |35|47|50|80|[3, 5, |, 4, 7, |, 5, 0, |, 8, 0, ]|
+---+------+---------+-----------+-----------------------------------+



In [25]:
df1.withColumn("marks_split",split(col("marks"),"[|]")).show(truncate=False)

+---+------+---------+-----------+----------------+
|id |name  |loc      |marks      |marks_split     |
+---+------+---------+-----------+----------------+
|1  |ravi  |patna    |34|45|55|65|[34, 45, 55, 65]|
|2  |ram   |bangalore|38|40|59|70|[38, 40, 59, 70]|
|3  |prasad|chennai  |37|42|54|77|[37, 42, 54, 77]|
|4  |sam   |pune     |36|46|52|76|[36, 46, 52, 76]|
|5  |shyam |delhi    |35|47|50|80|[35, 47, 50, 80]|
+---+------+---------+-----------+----------------+



In [26]:
df1.withColumn("marks_split",split(col("marks"),"\|")).show()

+---+------+---------+-----------+----------------+
| id|  name|      loc|      marks|     marks_split|
+---+------+---------+-----------+----------------+
|  1|  ravi|    patna|34|45|55|65|[34, 45, 55, 65]|
|  2|   ram|bangalore|38|40|59|70|[38, 40, 59, 70]|
|  3|prasad|  chennai|37|42|54|77|[37, 42, 54, 77]|
|  4|   sam|     pune|36|46|52|76|[36, 46, 52, 76]|
|  5| shyam|    delhi|35|47|50|80|[35, 47, 50, 80]|
+---+------+---------+-----------+----------------+



In [28]:
final_df= df1.withColumn("marks_split",split(col("marks"),"\\|"))\
            .withColumn("SUB1",col("marks_split")[0])\
            .withColumn("SUB2",col("marks_split")[1])\
            .withColumn("SUB3",col("marks_split")[2])\
            .withColumn("SUB4",col("marks_split")[3])\
            .drop("marks","marks_split")

final_df.show()

+---+------+---------+----+----+----+----+
| id|  name|      loc|SUB1|SUB2|SUB3|SUB4|
+---+------+---------+----+----+----+----+
|  1|  ravi|    patna|  34|  45|  55|  65|
|  2|   ram|bangalore|  38|  40|  59|  70|
|  3|prasad|  chennai|  37|  42|  54|  77|
|  4|   sam|     pune|  36|  46|  52|  76|
|  5| shyam|    delhi|  35|  47|  50|  80|
+---+------+---------+----+----+----+----+

